In [29]:
# Use RegEx to parse the tools from the llm response.

import json
from typing import Callable, Dict, Any, Literal, List, Optional, Union
from enum import Enum
import re

class Tool:
    def __init__(self, name: str, description: str, function: Callable):
        self.name = name
        self.description = description
        self.function = function
        
    def __call__(self, *args, **kwargs):
        return self.function(*args, **kwargs)

class ToolManager:
    def __init__(self):
        self.tools: Dict[str, Tool] = {}
        self.call_history: List[Dict] = []
        
    def register_tool(self, tool: Tool):
        """Register a new tool"""
        self.tools[tool.name] = tool
        
    def get_tool_descriptions(self) -> str:
        """Get formatted descriptions of all available tools"""
        descriptions = []
        for name, tool in self.tools.items():
            descriptions.append(f"Tool: {name}\nDescription: {tool.description}\n")
        return "\n".join(descriptions)
    
    def create_prompt(self, user_query: str) -> str:
        """Create a prompt that includes tool descriptions and usage instructions"""
        base_prompt = f"""Please help me accomplish the following task: {user_query}

Available tools:
{self.get_tool_descriptions()}

If you need to use a tool, respond in the following format:
<tool>
name: [tool name]
args: {{"arg1": "value1", "arg2": "value2"}}
</tool>

You can use multiple tools by providing multiple tool blocks, and reference the results of previous tools using the syntax {{result_N}}, where N is the index of the previous result (starting from 0).

If no tools are needed, respond normally.
Think step by step about whether tools are needed and how to use them effectively.
"""
        return base_prompt

    def extract_tool_calls(self, llm_response: str) -> List[Dict]:
        """Extract tool calls from LLM response"""
        tool_pattern = r'<tool>\s*name:\s*([^\n]+)\s*args:\s*({[^}]+})\s*</tool>'
        matches = re.findall(tool_pattern, llm_response)
        
        tool_calls = []
        for match in matches:
            try:
                name = match[0].strip()
                args = json.loads(match[1])
                tool_calls.append({"name": name, "args": args})
            except (json.JSONDecodeError, IndexError):
                continue
        return tool_calls    
    
    def execute_tool_calls(self, tool_calls: List[Dict]) -> List[str]:
        """Execute a list of tool calls and return their results"""
        results = []
        for i, call in enumerate(tool_calls):
            if call["name"] in self.tools:
                try:
                    # Replace result placeholders in arguments
                    for key, value in call["args"].items():
                        if isinstance(value, str) and value.startswith("{{result_"):
                            index = int(value[9:-2])
                            call["args"][key] = self.call_history[index]["result"]
                    result = self.tools[call["name"]](**call["args"])
                    self.call_history.append({"result": result})
                    results.append(f"Tool '{call['name']}' returned: {result}")
                except Exception as e:
                    results.append(f"Tool '{call['name']}' failed with error: {str(e)}")
            else:
                results.append(f"Tool '{call['name']}' not found")
        return results

def example_calculator(a: float, b: float, operation: str) -> float:
    """Example tool that performs basic arithmetic"""
    if operation == "add":
        return a + b
    elif operation == "multiply":
        return a * b
    raise ValueError(f"Unknown operation: {operation}")

# Example usage
def main():
    # Initialize tool manager
    manager = ToolManager()
    
    # Register example tool
    calculator = Tool(
        name="calculator",
        description="Performs basic arithmetic. Args: a (float), b (float), operation (str: 'add' or 'multiply')",
        function=example_calculator
    )
    manager.register_tool(calculator)
    
    # Example user query
    user_query = "Calculate 5 + 3 and then multiply the result by 2"
    
    # Create prompt for LLM
    prompt = manager.create_prompt(user_query)
    print("prompt send to llm: ")
    print(prompt)
    
    # Simulate LLM response (in real usage, this would come from your LLM)
    llm_response = """Let me help you with that calculation.

First, let's add 5 and 3:
<tool>
name: calculator
args: {"a": 5, "b": 3, "operation": "add"}
</tool>

Now, let's multiply the result by 2:
<tool>
name: calculator
args: {"a": {{result_0}}, "b": 2, "operation": "multiply"}
</tool>
"""
    
    # Extract and execute tool calls
    tool_calls = manager.extract_tool_calls(llm_response)
    results = manager.execute_tool_calls(tool_calls)
    
    # Print results
    print("\nResults:")
    for result in results:
        print(result)

if __name__ == "__main__":
    main()

prompt send to llm: 
Please help me accomplish the following task: Calculate 5 + 3 and then multiply the result by 2

Available tools:
Tool: calculator
Description: Performs basic arithmetic. Args: a (float), b (float), operation (str: 'add' or 'multiply')


If you need to use a tool, respond in the following format:
<tool>
name: [tool name]
args: {"arg1": "value1", "arg2": "value2"}
</tool>

You can use multiple tools by providing multiple tool blocks, and reference the results of previous tools using the syntax {result_N}, where N is the index of the previous result (starting from 0).

If no tools are needed, respond normally.
Think step by step about whether tools are needed and how to use them effectively.


Results:
Tool 'calculator' returned: 8


In [24]:
# Use pydantic to parse the tools

from pydantic import BaseModel, Field
import re
from typing import Any, Dict, List

class ToolArgs(BaseModel):
    a: Any
    b: Any
    operation: str

class Tool(BaseModel):
    name: str = Field(..., pattern="^calculator$")  # ensure name is "calculator"
    args: ToolArgs

class Document(BaseModel):
    tools: List[Tool] = []

    @classmethod
    def parse_text(cls, text: str) -> "Document":
        # Simple regex to split into tool blocks
        tool_blocks = re.findall(r'<tool>.*?</tool>', text, re.DOTALL)
        
        tools = []
        for block in tool_blocks:
            # Extract the content between tool tags
            content = block.replace('<tool>', '').replace('</tool>', '').strip()
            
            # Split into name and args sections
            name_part = re.search(r'name:\s*(.*?)\s*args:', content, re.DOTALL)
            args_part = re.search(r'args:\s*({.*})', content, re.DOTALL)
            
            if name_part and args_part:
                name = name_part.group(1).strip()
                # Convert string representation of args to dict
                args_str = args_part.group(1)
                # Handle the special case of {{result_0}}
                args_str = args_str.replace('{{result_0}}', '"{{result_0}}"')
                # Parse as Python dict
                import json
                try:
                    args_dict = json.loads(args_str)
                    tool = Tool(name=name, args=args_dict)
                    tools.append(tool)
                except json.JSONDecodeError as e:
                    print(f"Error parsing args JSON: {e}")
                    continue
        
        return cls(tools=tools)

# Test the implementation
text = '''Let me help you with that calculation.

First, let's add 5 and 3:
<tool>
name: calculator
args: {"a": 5, "b": 3, "operation": "add"}
</tool>

Now, let's multiply the result by 2:
<tool>
name: calculator
args: {"a": {{result_0}}, "b": 2, "operation": "multiply"}
</tool>'''

# Parse the document
doc = Document.parse_text(text)

# Print the results in a structured way
for i, tool in enumerate(doc.tools):
    print(f"\nTool {i + 1}:")
    print("Name:", tool.name)
    print("Args:", tool.args.model_dump())


Tool 1:
Name: calculator
Args: {'a': 5, 'b': 3, 'operation': 'add'}

Tool 2:
Name: calculator
Args: {'a': '{{result_0}}', 'b': 2, 'operation': 'multiply'}


In [26]:
# Adding descriptions
# use pydantic to handle tool parsing

from pydantic import BaseModel, Field, field_validator, model_validator
from typing import Callable, Any, List, Literal, Union
from enum import Enum

# Define tool types
class ToolType(str, Enum):
    CALCULATOR = "calculator"
    TEXT_PROCESSOR = "text_processor"

# Define different argument structures for each tool
class CalculatorArgs(BaseModel):
    a: Any
    b: Any
    operation: Literal["add", "multiply", "subtract", "divide"]

class TextProcessorArgs(BaseModel):
    text: str
    operation: Literal["uppercase", "lowercase", "capitalize"]

# Define the base tool and specific tool models
class BaseTool(BaseModel):
    name: ToolType
    args: Union[CalculatorArgs, TextProcessorArgs]
    description: str
    function: Callable

    @field_validator('args')
    @classmethod
    def validate_args_type(cls, v: Any, info: Any) -> Any:
        # In Pydantic v2, we use info.data to access other field values
        if 'name' in info.data:
            if info.data['name'] == ToolType.CALCULATOR and not isinstance(v, CalculatorArgs):
                raise ValueError("Calculator tool must use CalculatorArgs")
            if info.data['name'] == ToolType.TEXT_PROCESSOR and not isinstance(v, TextProcessorArgs):
                raise ValueError("Text processor tool must use TextProcessorArgs")
        return v

class Document(BaseModel):
    tools: List[BaseTool] = []

    @classmethod
    def parse_text(cls, text: str) -> "Document":
        import re
        tool_blocks = re.findall(r'<tool>.*?</tool>', text, re.DOTALL)
        
        tools = []
        for block in tool_blocks:
            content = block.replace('<tool>', '').replace('</tool>', '').strip()
            
            name_part = re.search(r'name:\s*(.*?)\s*args:', content, re.DOTALL)
            args_part = re.search(r'args:\s*({.*})', content, re.DOTALL)
            
            if name_part and args_part:
                name = name_part.group(1).strip()
                args_str = args_part.group(1)
                args_str = args_str.replace('{{result_0}}', '"{{result_0}}"')
                
                import json
                try:
                    args_dict = json.loads(args_str)
                    
                    # Create appropriate args model based on tool name
                    if name == ToolType.CALCULATOR:
                        args = CalculatorArgs.model_validate(args_dict)  # Using v2 validation
                    elif name == ToolType.TEXT_PROCESSOR:
                        args = TextProcessorArgs.model_validate(args_dict)  # Using v2 validation
                    else:
                        print(f"Unknown tool type: {name}")
                        continue
                    
                    tool = BaseTool.model_validate({"name": name, "args": args})  # Using v2 validation
                    tools.append(tool)
                except json.JSONDecodeError as e:
                    print(f"Error parsing args JSON: {e}")
                    continue
                except ValueError as e:
                    print(f"Error validating tool: {e}")
                    continue
        
        return cls.model_validate({"tools": tools})  # Using v2 validation

    def execute_tools(self) -> List[Any]:
        results = []
        for tool in self.tools:
            result = self.execute_tool(tool, results)
            results.append(result)
        return results

    def execute_tool(self, tool: BaseTool, previous_results: List[Any]) -> Any:
        if tool.name == ToolType.CALCULATOR:
            args = tool.args
            a = self._resolve_value(args.a, previous_results)
            b = self._resolve_value(args.b, previous_results)
            
            match args.operation:
                case "add":
                    return a + b
                case "multiply":
                    return a * b
                case "subtract":
                    return a - b
                case "divide":
                    return a / b
                
        elif tool.name == ToolType.TEXT_PROCESSOR:
            args = tool.args
            text = self._resolve_value(args.text, previous_results)
            
            match args.operation:
                case "uppercase":
                    return text.upper()
                case "lowercase":
                    return text.lower()
                case "capitalize":
                    return text.capitalize()
        
        raise ValueError(f"Unknown tool type or operation: {tool.name}")

    def _resolve_value(self, value: Any, previous_results: List[Any]) -> Any:
        if isinstance(value, str) and value.startswith("{{") and value.endswith("}}"):
            result_idx = int(value.strip("{}").split("_")[1])
            if result_idx < len(previous_results):
                return previous_results[result_idx]
        return value

    # Example of model-level validation in v2
    @model_validator(mode='after')
    def validate_tool_sequence(self) -> 'Document':
        # Add any document-level validation here
        return self

# Test the implementation
text = '''Let me help you with that calculation and text processing.

First, let's add 5 and 3:
<tool>
name: calculator
args: {"a": 5, "b": 3, "operation": "add"}
</tool>

Now, let's multiply the result by 2:
<tool>
name: calculator
args: {"a": {{result_0}}, "b": 2, "operation": "multiply"}
</tool>

Let's process some text:
<tool>
name: text_processor
args: {"text": "hello world", "operation": "uppercase"}
</tool>

And use the previous result:
<tool>
name: text_processor
args: {"text": "{{result_2}}", "operation": "lowercase"}
</tool>'''

# Parse and execute
doc = Document.parse_text(text)

# Print the tools and their results
results = doc.execute_tools()
for i, (tool, result) in enumerate(zip(doc.tools, results)):
    print(f"\nTool {i + 1}:")
    print("Name:", tool.name)
    print("Args:", tool.model_dump())  # Using v2 model_dump instead of dict()
    print("Result:", result)


Tool 1:
Name: ToolType.CALCULATOR
Args: {'name': <ToolType.CALCULATOR: 'calculator'>, 'args': {'a': 5, 'b': 3, 'operation': 'add'}}
Result: 8

Tool 2:
Name: ToolType.CALCULATOR
Args: {'name': <ToolType.CALCULATOR: 'calculator'>, 'args': {'a': '{{result_0}}', 'b': 2, 'operation': 'multiply'}}
Result: 16

Tool 3:
Name: ToolType.TEXT_PROCESSOR
Args: {'name': <ToolType.TEXT_PROCESSOR: 'text_processor'>, 'args': {'text': 'hello world', 'operation': 'uppercase'}}
Result: HELLO WORLD

Tool 4:
Name: ToolType.TEXT_PROCESSOR
Args: {'name': <ToolType.TEXT_PROCESSOR: 'text_processor'>, 'args': {'text': '{{result_2}}', 'operation': 'lowercase'}}
Result: hello world


In [27]:
# Adding operations and descriptions. 
# Use pydantic to do the tool parsing.

from pydantic import BaseModel, Field, field_validator, model_validator
from typing import Any, List, Literal, Union, Dict, Protocol, Callable
from enum import Enum
from abc import ABC, abstractmethod

# First define the base interfaces
class ToolOperation(Protocol):
    def execute(self, args: Any, context: Dict[str, Any]) -> Any:
        ...

class BaseToolArgs(BaseModel):
    operation: str

class BaseTool(ABC, BaseModel):
    name: str
    description: str
    args: BaseToolArgs

    @abstractmethod
    def execute(self, context: Dict[str, Any]) -> Any:
        pass

# Calculator Implementation
class CalculatorOperation:
    @staticmethod
    def add(a: float, b: float) -> float:
        return a + b

    @staticmethod
    def multiply(a: float, b: float) -> float:
        return a * b

    @staticmethod
    def subtract(a: float, b: float) -> float:
        return a - b

    @staticmethod
    def divide(a: float, b: float) -> float:
        return a / b

class CalculatorArgs(BaseToolArgs):
    a: Any
    b: Any
    operation: Literal["add", "multiply", "subtract", "divide"]

class CalculatorTool(BaseTool):
    name: Literal["calculator"]
    args: CalculatorArgs
    
    _operations: Dict[str, Callable] = {
        "add": CalculatorOperation.add,
        "multiply": CalculatorOperation.multiply,
        "subtract": CalculatorOperation.subtract,
        "divide": CalculatorOperation.divide,
    }

    @property
    def operation_descriptions(self) -> Dict[str, str]:
        return {
            "add": "Adds two numbers together",
            "multiply": "Multiplies two numbers",
            "subtract": "Subtracts second number from first",
            "divide": "Divides first number by second"
        }

    def execute(self, context: Dict[str, Any]) -> Any:
        # Resolve any result references
        a = self._resolve_value(self.args.a, context.get('previous_results', []))
        b = self._resolve_value(self.args.b, context.get('previous_results', []))
        
        operation = self._operations.get(self.args.operation)
        if not operation:
            raise ValueError(f"Unknown operation: {self.args.operation}")
        
        return operation(a, b)

    def _resolve_value(self, value: Any, previous_results: List[Any]) -> Any:
        if isinstance(value, str) and value.startswith("{{") and value.endswith("}}"):
            result_idx = int(value.strip("{}").split("_")[1])
            if result_idx < len(previous_results):
                return previous_results[result_idx]
        return value

# Text Processor Implementation
class TextProcessorOperation:
    @staticmethod
    def uppercase(text: str) -> str:
        return text.upper()

    @staticmethod
    def lowercase(text: str) -> str:
        return text.lower()

    @staticmethod
    def capitalize(text: str) -> str:
        return text.capitalize()

class TextProcessorArgs(BaseToolArgs):
    text: str
    operation: Literal["uppercase", "lowercase", "capitalize"]

class TextProcessorTool(BaseTool):
    name: Literal["text_processor"]
    args: TextProcessorArgs

    _operations: Dict[str, Callable] = {
        "uppercase": TextProcessorOperation.uppercase,
        "lowercase": TextProcessorOperation.lowercase,
        "capitalize": TextProcessorOperation.capitalize,
    }

    @property
    def operation_descriptions(self) -> Dict[str, str]:
        return {
            "uppercase": "Converts text to uppercase",
            "lowercase": "Converts text to lowercase",
            "capitalize": "Capitalizes the first letter of text"
        }

    def execute(self, context: Dict[str, Any]) -> Any:
        text = self._resolve_value(self.args.text, context.get('previous_results', []))
        
        operation = self._operations.get(self.args.operation)
        if not operation:
            raise ValueError(f"Unknown operation: {self.args.operation}")
        
        return operation(text)

    def _resolve_value(self, value: Any, previous_results: List[Any]) -> Any:
        if isinstance(value, str) and value.startswith("{{") and value.endswith("}}"):
            result_idx = int(value.strip("{}").split("_")[1])
            if result_idx < len(previous_results):
                return previous_results[result_idx]
        return value

# Tool Factory
class ToolFactory:
    _tools = {
        "calculator": CalculatorTool,
        "text_processor": TextProcessorTool,
    }

    @classmethod
    def create_tool(cls, name: str, args: Dict[str, Any], description: str = "") -> BaseTool:
        tool_cls = cls._tools.get(name)
        if not tool_cls:
            raise ValueError(f"Unknown tool type: {name}")
        
        return tool_cls(
            name=name,
            description=description or tool_cls.__doc__ or "No description available",
            args=args
        )

# Document class
class Document(BaseModel):
    tools: List[BaseTool] = []

    @classmethod
    def parse_text(cls, text: str) -> "Document":
        import re
        tool_blocks = re.findall(r'<tool>.*?</tool>', text, re.DOTALL)
        
        tools = []
        for block in tool_blocks:
            content = block.replace('<tool>', '').replace('</tool>', '').strip()
            
            name_part = re.search(r'name:\s*(.*?)\s*args:', content, re.DOTALL)
            args_part = re.search(r'args:\s*({.*})', content, re.DOTALL)
            
            if name_part and args_part:
                import json
                try:
                    name = name_part.group(1).strip()
                    args_dict = json.loads(args_part.group(1).replace('{{result_0}}', '"{{result_0}}"'))
                    
                    tool = ToolFactory.create_tool(name, args_dict)
                    tools.append(tool)
                except Exception as e:
                    print(f"Error creating tool: {e}")
                    continue
        
        return cls(tools=tools)

    def execute_tools(self) -> List[Any]:
        results = []
        context = {"previous_results": results}
        
        for tool in self.tools:
            try:
                result = tool.execute(context)
                results.append(result)
            except Exception as e:
                print(f"Error executing tool {tool.name}: {e}")
                results.append(None)
                
        return results

# Usage example
text = '''Let me help you with that calculation and text processing.

First, let's add 5 and 3:
<tool>
name: calculator
args: {"a": 5, "b": 3, "operation": "add"}
</tool>

Now, let's multiply the result by 2:
<tool>
name: calculator
args: {"a": {{result_0}}, "b": 2, "operation": "multiply"}
</tool>

Let's process some text:
<tool>
name: text_processor
args: {"text": "hello world", "operation": "uppercase"}
</tool>'''

# Parse and execute
doc = Document.parse_text(text)
results = doc.execute_tools()

# Print results
for i, (tool, result) in enumerate(zip(doc.tools, results)):
    print(f"\nTool {i + 1}:")
    print(f"Name: {tool.name}")
    print(f"Description: {tool.description}")
    print(f"Operation: {tool.args.operation}")
    print(f"Operation Description: {tool.operation_descriptions[tool.args.operation]}")
    print(f"Result: {result}")


Tool 1:
Name: calculator
Description: No description available
Operation: add
Operation Description: Adds two numbers together
Result: 8

Tool 2:
Name: calculator
Description: No description available
Operation: multiply
Operation Description: Multiplies two numbers
Result: 16

Tool 3:
Name: text_processor
Description: No description available
Operation: uppercase
Operation Description: Converts text to uppercase
Result: HELLO WORLD
